In [5]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt

# import util
from sort.sort import *
# from util import get_car, read_license_plate,
from utils.utils import get_car,read_license_plate

In [6]:
# LOADS THE MODELS
def load_models(car_detection_path,license_plate_detection_path):
    
    coco_model = YOLO(car_detection_path)
    license_plate_detector = YOLO(license_plate_detection_path)

    return coco_model,license_plate_detector

In [7]:
# Filter Vehicular Detections
def filter_vehicle_detections(detections):
    vehicle_classes = [2,3,5,7] # Car,bike,truck
    vehicle_detections = list()
    for detection in detections.boxes.data.tolist():
        x1,y1,x2,y2,conf_score,class_id = detection
        if(class_id in vehicle_classes): 
            vehicle_detections.append(detection) # Bounding boxes of the car detections

In [8]:
# Get License Plate text and Confidence Score
def get_license(license_plate,track_ids):
    x1,y1,x2,y2,conf_score,class_id = license_plate

    # Assign license plate to the car
    xcar1,ycar1,xcar2,ycar2,car_id = get_car(license_plate,track_ids) # Which license plate belongs to which car
    # Crop the License Plate
    license_plate_cropped = frame[int(y1):int(y2),int(x1):int(x2),:]
    # Process this License Plate (BGR)
    license_plate_grey = cv2.cvtColor(license_plate_cropped,cv2.COLOR_BGR2GRAY)
    # Thresholding
    _, license_plate_grey_thres = cv2.threshold(license_plate_grey, 64, 255, cv2.THRESH_BINARY_INV)
    #cv2.imshow('grey_threshold',license_plate_grey_thres)

    license_plate_text, license_plate_text_score = read_license_plate(license_plate_grey_thres)
    
    car_detection_values = (xcar1,ycar1,xcar2,ycar2,car_id)
    license_detection_values = (x1,y1,x2,y2,conf_score,class_id)

    return license_plate_text, license_plate_text_score,car_detection_values,license_detection_values

In [10]:
#______Change Script________

# Get the Models
car_detection_path = './models/yolov8n.pt'
license_plate_detection_path = './models/license_plate_detector.pt'
coco_model,license_plate_detector = load_models(car_detection_path,license_plate_detection_path)
mot_tracker = Sort()

# Read the Video
cap = cv2.VideoCapture('./data/sample.mp4')

results = {}
frame_nmr = -1
ret = True
while ret and frame_nmr<10:
    frame_nmr += 1
    ret, frame = cap.read()
    if ret:
        # Detect Vehicles
        detections = coco_model(frame)[0]
        vehicle_detections = filter_vehicle_detections(detections)
    # Track the vehicles using track ids
    track_ids = mot_tracker.update(np.asarray(vehicle_detections))
    # Detect License Plates
    license_plates = license_plate_detector(frame)[0]
    for license_plate in license_plates.boxes.data.tolist():
        license_plate_text, license_plate_text_score,car_id, car_detection_values,license_detection_values = get_license(license_plate,track_ids)
        xcar1,ycar1,xcar2,ycar2,car_id = (xcar1,ycar1,xcar2,ycar2,car_id)
        x1,y1,x2,y2,conf_score,class_id = license_detection_values 
        if(license_plate_text is not None):
            key = f'Frame:{frame_nmr}; CarID:{car_id}'

            results[key] = {

                'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                'license_plate': {'bbox': [x1, y1, x2, y2],
                'text': license_plate_text,
                'bbox_score': conf_score,
                'text_score': license_plate_text_score}

                }


0: 384x640 21 cars, 1 bus, 2 trucks, 101.8ms
Speed: 3.1ms preprocess, 101.8ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)


TypeError: len() of unsized object